# Monotonic
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/yggdrasil-decision-forests/blob/main/documentation/public/docs/tutorial/monotonic_feature.ipynb)


**Monotonic constraints** force a monotonic relationship between features and model predictions. For instance, we might want the model output to always increase when a specific feature value increases. Monotonicity is imposed with the `features` argument.

**Note:** Not all learners support monotonic constraints.

Let's train a model on the **Adult census** dataset with monotonic increasing constraints on features `age` and `hours_per_week`, i.e. let's force to model to predict increasing income with increasing age and increasing hours worked per week (all other features being constant).

In [1]:
import ydf
import pandas as pd

# Download a classification dataset and load it as a Pandas DataFrame.
ds_path = "https://raw.githubusercontent.com/google/yggdrasil-decision-forests/main/yggdrasil_decision_forests/test_data/dataset"
train_ds = pd.read_csv(f"{ds_path}/adult_train.csv")
test_ds = pd.read_csv(f"{ds_path}/adult_test.csv")

# Print the first 5 training examples
train_ds.head(5)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income
0,44,Private,228057,7th-8th,4,Married-civ-spouse,Machine-op-inspct,Wife,White,Female,0,0,40,Dominican-Republic,<=50K
1,20,Private,299047,Some-college,10,Never-married,Other-service,Not-in-family,White,Female,0,0,20,United-States,<=50K
2,40,Private,342164,HS-grad,9,Separated,Adm-clerical,Unmarried,White,Female,0,0,37,United-States,<=50K
3,30,Private,361742,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,<=50K
4,67,Self-emp-inc,171564,HS-grad,9,Married-civ-spouse,Prof-specialty,Wife,White,Female,20051,0,30,England,>50K


In [2]:
model = ydf.GradientBoostedTreesLearner(label="income",
                                features=[
                                    ydf.Feature("age", monotonic=+1),
                                    ydf.Feature("hours_per_week", monotonic=+1),
                                    ],
                                include_all_columns=True,
                                use_hessian_gain=True,
                                ).train(train_ds)

Train model on 22792 examples
Model trained in 0:00:02.326853


To verify that a model is monotonic with respect to a feature, we can examine the [Partial Dependence Plot](https://christophm.github.io/interpretable-ml-book/pdp.html) (PDP). In the PDP tab, the curves of `age` and `hours_per_week` are monotonically increasing.

In [3]:
model.analyze(test_ds, sampling=0.1)

For comparison, let's train the same model without the monotonic constraints and compare the partial dependence plots.

Here, the PDP of `age` and `hours_per_week` is not monotonic. For example, for ages greater than 60, the model output decreases as age increases.


In [4]:
model = ydf.GradientBoostedTreesLearner(label="income",
                                use_hessian_gain=True,
                                ).train(train_ds)

model.analyze(test_ds, sampling=0.1)

Train model on 22792 examples
Model trained in 0:00:02.118608
